In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import layers, utils, init, learn, explore
from tfomics import neuralnetwork as nn
from tfomics.build_network import *
from model_zoo import tim_model

In [2]:
filename = 'processed_dataset.hdf5'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf
y_train = ltf
X_valid = dtf_crossval
y_valid = ltf_crossval


CPU times: user 328 ms, sys: 584 ms, total: 912 ms
Wall time: 909 ms


In [3]:

# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  

# load model
net, placeholders, optimization = tim_model.model(input_shape, num_labels)

# build neural network class
nnmodel = nn.NeuralNet(net, [placeholders['inputs']])
nnmodel.inspect_layers()

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, placeholders, optimization, save='best', filepath='test')

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: input
(?, 3, 330, 1)
layer2: conv1
(?, 2, 326, 18)
layer3: conv1_batch
(?, 2, 326, 18)
layer4: conv1_active
(?, 2, 326, 18)
layer5: conv2
(?, 1, 322, 40)
layer6: conv2_batch
(?, 1, 322, 40)
layer7: conv2_active
(?, 1, 322, 40)
layer8: conv2_pool
(?, 1, 32, 40)
layer9: conv3
(?, 1, 32, 15)
layer10: conv3_batch
(?, 1, 32, 15)
layer11: conv3_active
(?, 1, 32, 15)
layer12: dense1
(?, 100)
layer13: dense1_bias
(?, 100)
layer14: dense1_active
(?, 100)
layer15: dense1_dropout
(?, 100)
layer16: dense2
(?, 2)
layer17: dense2_bias
(?, 2)
layer18: dense2_active
(?, 2)
layer19: output
(?, 2)
----------------------------------------------------------------------------


In [ ]:
# run session
sess = tf.Session()

# initialize variables
sess.run(tf.initialize_all_variables())

In [ ]:

X = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
X2 = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}
data = {'train': X, 'valid': X2}
learn.train_minibatch(sess, nntrainer, data, batch_size=128, num_epochs=500, 
                    patience=10, verbose=1, shuffle=True)